In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
file = open('/blue/bsc4452/share/Class_Files/data/flights.May2020-April2022.csv')
columns = ['FL_DATE', 'ORIGIN', 'DEST', 'ARR_DELAY_NEW']
df = pd.read_csv(file, usecols=columns)

#convert dates to weekdays
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], format='%m/%d/%Y %H:%M:%S AM')
df['FL_DATE'] = df['FL_DATE'].dt.day_name()

#function to get average delays for each weekday
def weekdayavg(airport):
    global Weekday
    global Avg_Delay
    global daydict
    global daydf
    monavg = tueavg = wedavg = thuavg = friavg = satavg = sunavg = None
    Weekday = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    daydict = {}
    mon = df[(df.ORIGIN==airport) | (df.DEST==airport) & (df.FL_DATE=='Monday')]
    monavg = mon['ARR_DELAY_NEW'].mean()
    tue = df[(df.ORIGIN==airport) | (df.DEST==airport) & (df.FL_DATE=='Tuesday')]
    tueavg = tue['ARR_DELAY_NEW'].mean()
    wed = df[(df.ORIGIN==airport) | (df.DEST==airport) & (df.FL_DATE=='Wednseday')]
    wedavg = wed['ARR_DELAY_NEW'].mean()
    thu = df[(df.ORIGIN==airport) | (df.DEST==airport) & (df.FL_DATE=='Thursday')]
    thuavg = thu['ARR_DELAY_NEW'].mean()
    fri = df[(df.ORIGIN==airport) | (df.DEST==airport) & (df.FL_DATE=='Friday')]
    friavg = fri['ARR_DELAY_NEW'].mean()
    sat = df[(df.ORIGIN==airport) | (df.DEST==airport) & (df.FL_DATE=='Saturday')]
    satavg = sat['ARR_DELAY_NEW'].mean()
    sun = df[(df.ORIGIN==airport) | (df.DEST==airport) & (df.FL_DATE=='Sunday')]
    sunavg = sun['ARR_DELAY_NEW'].mean()
    Avg_Delay = [monavg, tueavg, wedavg, thuavg, friavg, satavg, sunavg]
    daydict = {Weekday[i]:Avg_Delay[i] for i in range(len(Weekday))}
    
#filters nan values from weekday dictionary    
def filterd(d):
    global filterdict
    filterdict = {}
    for key, value in d.items():
        if type(value) == float:
            filterdict[key] = value
    filterdict = {k: filterdict[k] for k in filterdict if not pd.isna(filterdict[k])}
    keys = list(filterdict.keys())
    values = list(filterdict.values())
    filterdict = {'Weekday':keys, 'Avg_Delay':values}

#creates list of all airport codes in the data    
airports = pd.unique(df[['ORIGIN', 'DEST']].values.ravel())
airports = airports.tolist()

#creating bar graphs and writing raw data to a file
#limited iterations because it takes a while and makes a lot of jpegs
i = 0
for item in airports:
    weekdayavg(item)
    filterd(daydict)
    filterdf = pd.DataFrame(filterdict)
    with open('weekdayflightrawdata.txt', 'a') as data:
        data.write(item + ': ' + str(filterdict) + '\n')
    plot = filterdf.plot.bar(x='Weekday', y='Avg_Delay', title=item, legend=False)
    plot.bar_label(plot.containers[0])
    plot.set_ylabel("Average Delay (minutes)")
    plot.set_xlabel("Weekday")
    fig = plot.figure
    fig.savefig(item + '_Weekday_Delays.jpg')
    i += 1
    if i > 4:
        break